In [ ]:
# import data. Originally used method found on github by rasbt,
# but further expanded using python machine learning v2 pages 380-400
import os
import sys
import tarfile
import time
import pandas as pd

source = 'http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz'
target = 'aclImdb_v1.tar.gz'


def reporthook(count, block_size, total_size):
    global start_time
    if count == 0:
        start_time = time.time()
        return
    duration = time.time() - start_time
    progress_size = int(count * block_size)
    speed = progress_size / (1024.**2 * duration)
    percent = count * block_size * 100. / total_size
    sys.stdout.write("\r%d%% | %d MB | %.2f MB/s | %d sec elapsed" %
                    (percent, progress_size / (1024.**2), speed, duration))
    sys.stdout.flush()


if not os.path.isdir('aclImdb') and not os.path.isfile('aclImdb_v1.tar.gz'):
    
    if (sys.version_info < (3, 0)):
        import urllib
        urllib.urlretrieve(source, target, reporthook)
    
    else:
        import urllib.request
        urllib.request.urlretrieve(source, target, reporthook)
        
        
if not os.path.isdir('aclImdb'):

    with tarfile.open(target, 'r:gz') as tar:
        tar.extractall()

# change the basepath to the directory of the
# unzipped movie dataset

basepath = 'aclImdb'

labels = {'pos': 1, 'neg': 0}
df = pd.DataFrame()
for s in ('test', 'train'):
    for l in ('pos', 'neg'):
        path = os.path.join(basepath, s, l)
        for file in sorted(os.listdir(path)):
            with open(os.path.join(path, file), 
                      'r', encoding='utf-8') as infile:
                txt = infile.read()
            df = df.append([[txt, labels[l]]], 
                           ignore_index=True)
df.columns = ['review', 'sentiment']

df.head()

review  sentiment
0  I went and saw this movie last night after bei...          1
1  Actor turned director Bill Paxton follows up h...          1
2  As a recreational golfer with some knowledge o...          1
3  I saw this film in a sneak preview, and it is ...          1
4  Bill Paxton has taken the true story of the 19...          1

In [ ]:
# format data into a dataframe 
import numpy as np
np.random.seed(0)
df = df.reindex(np.random.permutation(df.index))
df.to_csv('movie_data.csv', index=False, encoding='utf-8')
df = pd.read_csv('movie_data.csv', encoding='utf-8')
df.head()

review  sentiment
0  Election is a Chinese mob movie, or triads in ...          1
1  I was just watching a Forensic Files marathon ...          0
2  Police Story is a stunning series of set piece...          1
3  Dear Readers,<br /><br />The final battle betw...          1
4  I have seen The Perfect Son about three times....          1

In [ ]:
#Step 1 on homework
#clean data to remove emojis, random figures, and unuseful words
import re
def clean(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)',
                           text)
    text = (re.sub('[\W]+', ' ', text.lower()) +
            ' '.join(emoticons).replace('-', ''))
    return text
  
"""Applys the clean function to each review in the data frame."""
df['review'] = df['review'].apply(clean)

df.head()

review  sentiment
0  election is a chinese mob movie or triads in t...          1
1  i was just watching a forensic files marathon ...          0
2  police story is a stunning series of set piece...          1
3  dear readers the final battle between the rebe...          1
4  i have seen the perfect son about three times ...          1

In [ ]:
# Step 2 on homework
# tokenize and count words
# even using the RAM optimizing processor, my session would crash off 50,000 values
# when trying to do step 3 of the assignment. 10,000 values allowed the program to work

from sklearn.feature_extraction.text import CountVectorizer

docs = df['review'][:5000].values
count = CountVectorizer()

bag = count.fit_transform(docs)

words = count.vocabulary_

print(words)

{'election': 11145, 'is': 18299, 'chinese': 6160, 'mob': 22556, 'movie': 22982, 'or': 24491, 'triads': 35624, 'in': 17409, 'this': 34809, 'case': 5548, 'every': 11963, 'two': 35960, 'years': 38672, 'an': 1575, 'held': 16077, 'to': 35100, 'decide': 8843, 'on': 24368, 'new': 23632, 'leader': 19896, 'and': 1616, 'at': 2417, 'first': 13076, 'it': 18346, 'seems': 30532, 'toss': 35283, 'up': 36725, 'between': 3617, 'big': 3679, 'tony': 35202, 'leung': 20120, 'ka': 18928, 'fai': 12438, 'as': 2253, 'know': 19375, 'him': 16331, 'the': 34686, 'other': 24626, 'lok': 20509, 'simon': 31405, 'yam': 38622, 'who': 38034, 'was': 37673, 'judge': 18821, 'full': 13934, 'contact': 7459, 'though': 34833, 'once': 24369, 'wins': 38205, 'refuses': 28308, 'accept': 645, 'choice': 6186, 'goes': 14756, 'whatever': 37941, 'lengths': 20041, 'he': 15932, 'can': 5258, 'secure': 30495, 'recognition': 28115, 'unlike': 36505, 'any': 1858, 'asian': 2280, 'film': 12963, 'watch': 37707, 'featuring': 12719, 'gangsters': 141

In [ ]:
# Step 3 on homework
# get rid of irrelevent words that appear across multiple documents from both classes

from sklearn.feature_extraction.text import TfidfTransformer

np.set_printoptions(precision=2)

tfidf = TfidfTransformer(use_idf=True, 
                         norm='l2', 
                         smooth_idf=True)

print(tfidf.fit_transform(count.fit_transform(docs))
      .toarray())

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [ ]:
# tokenize 
# compare the two functions to see if different tokenizations change the sentiment

# decided to just use str.split as my tokenizer to speed up runtimes

# from nltk.stem.porter import PorterStemmer

# porter = PorterStemmer()

# def tokens(text):
#   return text.split()

# def tokensPorter(text):
#   return [porter.stem(word) for word in text.split()]

In [ ]:
# divide dataframe of cleaned text docs into training and testing
X_train = df.loc[:25000, 'review'].values
y_train = df.loc[:25000, 'sentiment'].values
X_test = df.loc[25000:, 'review'].values
y_test = df.loc[25000:, 'sentiment'].values


In [ ]:
# bring in stop words
from nltk.corpus import stopwords
stop = stopwords.words('english')

In [ ]:
# use gridsearchcv to find optimal set of params for logistic regression
# train model using 5-fold cross validation
# I used sklearn packages to build my model as shown in 

from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer


tfidf = TfidfVectorizer(strip_accents=None, lowercase=False, preprocessor=None)

# removed the other parameters except the ones I want
param_grid = [{'vect__ngram_range': [(1,1)], 'vect__stop_words': [stop, None],'vect__tokenizer': [str.split], 'clf__penalty': ['l2'], 'clf__C': [1.0, 10.0, 100.0]}]

# increased max_iterations to be able to run
# also had to clear my 
lr_tfidf = Pipeline([('vect', tfidf),('clf', LogisticRegression(random_state=0, max_iter=10000))])

gs_lr_tfidf = GridSearchCV(lr_tfidf, param_grid, scoring='accuracy',cv=5, verbose=1,n_jobs=-1)

gs_lr_tfidf.fit(X_train, y_train)

Fitting 5 folds for each of 6 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:  1.5min finished


GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        TfidfVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.float64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=False,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                                        n

In [ ]:
#print accuracy of best model
print('CV Accuracy: %.3f' % gs_lr_tfidf.best_score_)
# CV Accuracy: 0.899
clf = gs_lr_tfidf.best_estimator_
print('Test Accuracy: %.3f'% clf.score(X_test, y_test))
# Test Accuracy: 0.896


CV Accuracy: 0.899
Test Accuracy: 0.896


In [ ]:
# # the rest of the code from here on are previous attempts before I looked at the book
# # I used github user rasbt as a guidline, later to find he was one of the writers of our book
# X_train = df.loc[:2500, 'review'].values
# y_train = df.loc[:2500, 'sentiment'].values

# X_test = df.loc[2500:, 'review'].values
# y_test = df.loc[2500:, 'sentiment'].values

# search.fit(X_train, y_train)

In [ ]:
# # this was another attempt at building a log reg for the imdb dataset
# # I used Aaron Kub's lesson found at  https://towardsdatascience.com/sentiment-analysis-with-python-part-1-5ce197074184
# # and implemented as he did
# !pip install mglearn
# from sklearn.feature_extraction.text import CountVectorizer
# from sklearn.linear_model import LogisticRegression
# from sklearn.datasets import load_files
# from sklearn.model_selection import GridSearchCV
# import numpy as np
# import mglearn
# import matplotlib.pyplot as plt

# reviews_train = load_files("aclImdb/train/")
# text_train, y_train = reviews_train.data, reviews_train.target

# print("Number of documents in train data: {}".format(len(text_train)))
# print("Samples per class (train): {}".format(np.bincount(y_train)))

# reviews_test = load_files("aclImdb/test/")
# text_test, y_test = reviews_test.data, reviews_test.target

# print("Number of documents in test data: {}".format(len(text_test)))
# print("Samples per class (test): {}".format(np.bincount(y_test)))


# vect = CountVectorizer(min_df=5, ngram_range=(2, 2))
# X_train = vect.fit(text_train).transform(text_train)
# X_test = vect.transform(text_test)

# print("Vocabulary size: {}".format(len(vect.vocabulary_)))
# print("X_train:\n{}".format(repr(X_train)))
# print("X_test: \n{}".format(repr(X_test)))

# feature_names = vect.get_feature_names()
# print("Number of features: {}".format(len(feature_names)))

# param_grid = {'C': [0.001, 0.01, 0.1, 1, 10]}
# grid = GridSearchCV(LogisticRegression(), param_grid, cv=5)
# grid.fit(X_train, y_train)

# print("Best cross-validation score: {:.2f}".format(grid.best_score_))
# print("Best parameters: ", grid.best_params_)
# print("Best estimator: ", grid.best_estimator_)

# mglearn.tools.visualize_coefficients(grid.best_estimator_.coef_, feature_names, n_top_features=25)
# plt.show()

# lr = grid.best_estimator_
# lr.predict(X_test)
# print("Score: {:.2f}".format(lr.score(X_test, y_test)))